In [11]:
# Define two variants of the prompt to test zero-shot
# vs few-shot:
prompt_A = """Product description: A pair of shoes that can
fit any foot size.
Seed words: adaptable, fit, omni-fit.
Product names:"""

prompt_B = """Product description: A home milkshake maker.
Seed words: fast, healthy, compact.
Product names: HomeShaker, Fit Shaker, QuickShake, Shake
Maker

Product description: A watch that can tell accurate time in
space.
Seed words: astronaut, space-hardened, eliptical orbit
Product names: AstroTime, SpaceGuard, Orbit-Accurate,
EliptoTime.

Product description: A pair of shoes that can fit any foot
size.
Seed words: adaptable, fit, omni-fit.
Product names:"""

test_prompts = [prompt_A, prompt_B]

import pandas as pd
from openai import OpenAI
import os

# Set your OpenAI key as an environment variable
# https://platform.openai.com/api-keys
client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY'],  # Default
)

def get_response(prompt):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant."
            },
            {
                "role": "user",
                "content": prompt
            }
        ]
    )
    return response.choices[0].message.content

# Iterate through the prompts and get responses
responses = []
num_tests = 5

for idx, prompt in enumerate(test_prompts):
    # prompt number as a letter
    var_name = chr(ord('A') + idx)

    for i in range(num_tests):
        # Get a response from the model
        response = get_response(prompt)

        data = {
            "variant": var_name,
            "prompt": prompt, 
            "response": response
            }
        responses.append(data)

# Convert responses into a dataframe
df = pd.DataFrame(responses)

# Save the dataframe as a CSV file
df.to_csv("responses.csv", index=False)

print(df)

  variant                                             prompt  \
0       A  Product description: A pair of shoes that can\...   
1       A  Product description: A pair of shoes that can\...   
2       A  Product description: A pair of shoes that can\...   
3       A  Product description: A pair of shoes that can\...   
4       A  Product description: A pair of shoes that can\...   
5       B  Product description: A home milkshake maker.\n...   
6       B  Product description: A home milkshake maker.\n...   
7       B  Product description: A home milkshake maker.\n...   
8       B  Product description: A home milkshake maker.\n...   
9       B  Product description: A home milkshake maker.\n...   

                                            response  
0  1. OmniFit Shoes\n2. AdaptFit Footwear\n3. Omn...  
1  1. AdaptaFit Shoes\n2. OmniFit Footwear\n3. Fl...  
2  1. AdaptFit Shoes\n2. FlexiFit Footwear\n3. Om...  
3  1. Adaptafit Shoes\n2. Omni-Fit Shoes\n3. Flex...  
4  1. OmniFit Shoes\

In [12]:
import ipywidgets as widgets
from IPython.display import display
import pandas as pd

# load the responses.csv file
df = pd.read_csv("responses.csv")

# Shuffle the dataframe
df = df.sample(frac=1).reset_index(drop=True)

# df is your dataframe and 'response' is the column with the 
# text you want to test
response_index = 0
# add a new column to store feedback
df['feedback'] = pd.Series(dtype='str') 

def on_button_clicked(b):
    global response_index
    #  convert thumbs up / down to 1 / 0
    user_feedback = 1 if b.description == "\U0001F44D" else 0

    # update the feedback column
    df.at[response_index, 'feedback'] = user_feedback

    response_index += 1
    if response_index < len(df):
        update_response()
    else:
        # save the feedback to a CSV file
        df.to_csv("results.csv", index=False)

        print("A/B testing completed. Here's the results:")
        # Calculate score and num rows for each variant
        summary_df = df.groupby('variant').agg(
            count=('feedback', 'count'), 
            score=('feedback', 'mean')).reset_index()
        print(summary_df)
        
def update_response():
    new_response = df.iloc[response_index]['response']
    if pd.notna(new_response):
        new_response = "<p>" + new_response + "</p>"
    else:
        new_response = "<p>No response</p>"
    response.value = new_response
    count_label.value = f"Response: {response_index + 1}"
    count_label.value += f"/{len(df)}"

response = widgets.HTML()
count_label = widgets.Label()

update_response()

thumbs_up_button = widgets.Button(description='\U0001F44D')
thumbs_up_button.on_click(on_button_clicked)

thumbs_down_button = widgets.Button(
    description='\U0001F44E')
thumbs_down_button.on_click(on_button_clicked)

button_box = widgets.HBox([thumbs_down_button, 
thumbs_up_button])

display(response, button_box, count_label)

HTML(value='<p>AdaptaFit, OmniFit, PerfectFit, FlexiFit</p>')

Label(value='Response: 1/10')

A/B testing completed. Here's the results:
  variant  count score
0       A      5   0.6
1       B      5   0.8
